In [40]:
from google.colab import drive
drive.mount('/content/gdrive/')

path = '/content/gdrive/My Drive/__학교/알고리즘/팀플/JgDiff/'

import sys
sys.path.append(path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LinearRegression, Ridge, LogisticRegression
from sklearn.kernel_ridge import KernelRidge
sys.path.insert(0, '..')
import bigdata

In [0]:
def trainSet(t):
    return t[t['matchId'].apply(lambda x: int(x)%5!=0)]

def testSet(t):
    return t[t['matchId'].apply(lambda x: int(x)%5==0)]

In [0]:
champions = pd.read_csv(path+"./supplement/champions.csv")
champions = champions.reset_index().set_index('id')

In [0]:
matchroles = pd.read_csv(path+"/supplement/matchrole.csv")
matchroles_tr = trainSet(matchroles).reset_index(drop=True)
matchroles_ts = testSet(matchroles).reset_index(drop=True)

In [0]:
kills = pd.read_csv(path+"/supplement/kills.csv")
kills_tr = trainSet(kills).reset_index(drop=True)
kills_ts = testSet(kills).reset_index(drop=True)

In [0]:
firstkills = kills.groupby('matchId').first()

# Existence of Invasion

In [45]:
# kill around the first buff
invaded_tr = kills_tr.groupby('matchId').first()['timestamp'] < 105000
invaded_ts = kills_ts.groupby('matchId').first()['timestamp'] < 105000

invaded_tr = invaded_tr[matchroles_tr['matchId']].fillna(False)
invaded_ts = invaded_ts[matchroles_ts['matchId']].fillna(False)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1146: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [46]:
matchroles.set_index('matchId').iloc[:,11:]

,Top_0_ch,Jgl_0_ch,Mid_0_ch,Bot_0_ch,Sup_0_ch,Top_1_ch,Jgl_1_ch,Mid_1_ch,Bot_1_ch,Sup_1_ch
matchId,,,,,,,,,,
3273218631,122,56,30,18,12,133,164,34,8,9
3273221800,80,163,4,81,201,79,107,55,202,25
3278904898,58,28,4,106,25,30,141,55,202,26
3287833940,27,141,266,202,555,36,35,126,61,53
3292739224,27,164,103,81,25,126,141,3,145,16
...,...,...,...,...,...,...,...,...,...,...
3956480499,126,11,44,67,117,31,64,136,145,40
3956483153,126,76,268,22,25,85,64,127,110,16
3956483610,240,30,84,18,89,114,64,8,145,412


In [47]:
champions.loc[matchroles.set_index('matchId').iloc[:,11+0*5:11+0*5+5].loc[[3273221800,3305223985]].values.reshape(-1)]['index'].values

array([ 85, 112, 120,  26,  13,  15,  35,  52,  26,  86])

In [0]:
def champ_onehot(match, team):
    a = np.zeros([len(match), len(champions)])
    for i in range(len(match)):
        a[i][champions.loc[matchroles.set_index('matchId').iloc[:,11+team*5:11+team*5+5].loc[match[i]].values.reshape(-1)]['index']] = 1
    return a

In [0]:
tr_matchids = matchroles_tr['matchId']
ts_matchids = matchroles_ts['matchId']

X_tr = np.concatenate([champ_onehot(tr_matchids, 0),champ_onehot(tr_matchids, 1)], axis=1)
X_ts = np.concatenate([champ_onehot(ts_matchids, 0),champ_onehot(ts_matchids, 1)], axis=1)

In [51]:
print(tr_matchids)

Int64Index([3273218631, 3278904898, 3292739224, 3292786341, 3305077757,
            3305168152, 3305241992, 3415011667, 3415206653, 3415334197,
            ...
            3956405482, 3956416007, 3956430832, 3956441503, 3956441753,
            3956445599, 3956461246, 3956480499, 3956483153, 3956497316],
           dtype='int64', name='matchId', length=14620)


In [52]:
linear = LinearRegression().fit(X_tr, invaded_tr.astype(int))
print(np.mean(invaded_tr == (linear.predict(X_tr)  > 0.5)))
print(np.mean(invaded_ts == (linear.predict(X_ts)  > 0.5)))

0.8104651162790698
0.8195848855774348


In [53]:
ridge = Ridge().fit(X_tr, invaded_tr.astype(int))
print(np.mean(invaded_tr == (ridge.predict(X_tr)  > 0.5)))
print(np.mean(invaded_ts == (ridge.predict(X_ts)  > 0.5)))

0.8105335157318742
0.8195848855774348


In [54]:
lasso = Lasso().fit(X_tr, invaded_tr.astype(int))
print(np.mean(invaded_tr == (lasso.predict(X_tr)  > 0.5)))
print(np.mean(invaded_ts == (lasso.predict(X_ts)  > 0.5)))

0.8105335157318742
0.8195848855774348


In [55]:
logistic = LogisticRegression().fit(X_tr, invaded_tr.astype(int))
print(np.mean(invaded_tr == (logistic.predict(X_tr)  > 0.5)))
print(np.mean(invaded_ts == (logistic.predict(X_ts)  > 0.5)))

0.8099863201094392
0.819318786588611


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [56]:
np.mean(logistic.predict(X_tr))

0.0013679890560875513

Linear regression 'cheats' by predicting everything to be 0... This is not good!

In [61]:
kernel = KernelRidge().fit(X_tr, invaded_tr.astype(int))
print(np.mean(invaded_tr == (kernel.predict(X_tr)  > 0.5)))
print(np.mean(invaded_ts == (kernel.predict(X_ts)  > 0.5)))

0.8105335157318742
0.8195848855774348


In [0]:
X2_tr = champ_onehot(tr_matchids, 0)+champ_onehot(tr_matchids, 1)
X2_ts = champ_onehot(ts_matchids, 0)+champ_onehot(ts_matchids, 1)

In [63]:
linear2 = LinearRegression().fit(X2_tr, invaded_tr.astype(int))
print(np.mean(invaded_tr == (linear2.predict(X2_tr)  > 0.5)))
print(np.mean(invaded_ts == (linear2.predict(X2_ts)  > 0.5)))

0.8105335157318742
0.8195848855774348


In [64]:
logistic2 = LogisticRegression().fit(X2_tr, invaded_tr.astype(int))
print(np.mean(invaded_tr == (logistic2.predict(X2_tr)  > 0.5)))
print(np.mean(invaded_ts == (logistic2.predict(X2_ts)  > 0.5)))

0.8104651162790698
0.819318786588611


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
def sup_champ_onehot(match, team):
    a = np.zeros([len(match), len(champions)])
    for i in range(len(match)):
        a[i][champions.loc[matchroles.set_index('matchId').iloc[:,11+team*5+4].loc[match[i]]]['index']] = 1
    return a

In [0]:
X3_tr = sup_champ_onehot(tr_matchids, 0)+sup_champ_onehot(tr_matchids, 1)
X3_ts = sup_champ_onehot(ts_matchids, 0)+sup_champ_onehot(ts_matchids, 1)

## Good ol' conditional probability

In [78]:
print(champions[champions.name=='Blitzcrank'])
print(champions[champions.name=='Pyke'])

    index        name
id                   
53     11  Blitzcrank
     index  name
id              
555     87  Pyke


In [0]:
print(np.sum((invaded_tr.values*X2_tr.T).T, axis=0) / np.sum(X2_tr, axis=0))

champions = champions.set_index('index')
champions['invade_prob'] = np.sum((invaded_tr.values*X2_tr.T).T, axis=0) / np.sum(X2_tr, axis=0)
champions = champions.set_index('id')
champions

In [101]:
champions.sort_values('invade_prob', ascending=False).values

array([['Master Yi', 0.2863070539419087],
       ['Ziggs', 0.26785714285714285],
       ['Anivia', 0.25951557093425603],
       ['Tristana', 0.2528548123980424],
       ['Volibear', 0.25252525252525254],
       ['Singed', 0.25139664804469275],
       ['Trundle', 0.24324324324324326],
       ['Olaf', 0.24284806102987921],
       ['Jhin', 0.2361111111111111],
       ['Twitch', 0.23529411764705882],
       ['Blitzcrank', 0.23478655767484105],
       ['Karthus', 0.22423849643551522],
       ['Teemo', 0.22419928825622776],
       ['Tryndamere', 0.22413793103448276],
       ['Rammus', 0.22302158273381295],
       ['Taric', 0.22127659574468084],
       ['Gnar', 0.21893491124260356],
       ['Corki', 0.21839080459770116],
       ["Cho'Gath", 0.2175925925925926],
       ['Morgana', 0.215633423180593],
       ['Nunu & Willump', 0.21543985637342908],
       ['Xin Zhao', 0.2146814404432133],
       ['Kayn', 0.21382289416846653],
       ['Shen', 0.21303656597774245],
       ['Urgot', 0.212121212121

In [108]:
print(np.sum(X2_tr, axis=0)[champions[champions.name=='Anivia'].index])
print(np.sum(X2_tr, axis=0)[champions[champions.name=='Ziggs'].index])
print(np.sum(X2_tr, axis=0)[champions[champions.name=='Master Yi'].index])
print(np.sum(X2_tr, axis=0)[champions[champions.name=='Tristana'].index])
print(np.sum(X2_tr, axis=0)[champions[champions.name=='Volibear'].index])
print(np.sum(X2_tr, axis=0)[champions[champions.name=='Blitzcrank'].index])

[289.]
[56.]
[241.]
[613.]
[495.]
[2202.]


# Alternative Definition of 'Invasion'

In [0]:
playerFrame = pd.read_csv(path+"/supplement/playerFrameTable.csv")


In [127]:
x = bigdata.diagTransform(playerFrame[['position_x', 'position_y']].values)[:,0]
team = (playerFrame['participantId'].astype(int) > 5).astype(int)
linepassed = (-1+2*team)*x < 0

playerFrame['linepassed'] = linepassed
playerFrame[playerFrame['frame'] == 1]


,Unnamed: 0,matchId,frame,participantKey,participantId,position_x,position_y,currentGold,totalGold,level,xp,linepassed
10,10,3.273219e+09,1.0,1.0,1.0,7983.0,2176.0,0.0,500.0,1.0,0.0,False
11,11,3.273219e+09,1.0,2.0,2.0,8014.0,1780.0,0.0,500.0,1.0,0.0,False
12,12,3.273219e+09,1.0,3.0,3.0,362.0,135.0,0.0,500.0,1.0,0.0,False
13,13,3.273219e+09,1.0,4.0,4.0,3311.0,8976.0,0.0,500.0,1.0,0.0,False
14,14,3.273219e+09,1.0,5.0,5.0,4861.0,7422.0,0.0,500.0,1.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4459085,4459085,3.956497e+09,1.0,6.0,6.0,7430.0,11174.0,5.0,505.0,1.0,0.0,False
4459086,4459086,3.956497e+09,1.0,7.0,7.0,6940.0,11345.0,0.0,500.0,1.0,0.0,False
4459087,4459087,3.956497e+09,1.0,8.0,8.0,14340.0,14390.0,500.0,500.0,1.0,0.0,False
4459088,4459088,3.956497e+09,1.0,9.0,10.0,10059.0,3911.0,0.0,500.0,1.0,0.0,True


## Neural Network

In [0]:
import torch